In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import re

In [ ]:
np.set_printoptions(suppress=True)
pd.set_option('display.max_columns',8000)
pd.set_option('display.max_rows',7000)

In [ ]:
from google.colab import files
files.upload()

Saving test_oJQbWVk.csv to test_oJQbWVk.csv


{'test_oJQbWVk.csv': b'id,tweet\r\n7921,I hate the new #iphone upgrade. Won\'t let me download apps. #ugh #apple sucks\r\n7922,currently shitting my fucking pants. #apple #iMac #cashmoney #raddest #swagswagswag http://instagr.am/p/UUIS0bIBZo/\r\n7923,"I\'d like to puts some CD-ROMS on my iPad, is that possible?\' \xe2\x80\x94 Yes, but wouldn\'t that block the screen?\n"\r\n7924,"My ipod is officially dead. I lost all my pictures and videos from the 1D and 5sos concert,and from Vet Camp #hatinglife #sobbing"\r\n7925,Been fighting iTunes all night! I only want the music I $&@*# paid for\r\n7926,#Repost @getbakednfried with repostapp \xe3\x83\xbb\xe3\x83\xbb\xe3\x83\xbb to announce we will have our #apple #bourbon\xe2\x80\xa6 https://www.instagram.com/p/BK5okHOgWSS/\r\n7927,This new apple software update is really doing things to my phone. Bad things apple\r\n7928,BABY #iPhone #iphone6s #gold #new #apple #appleisbest #64GB #geg\xc3\xb6nnt #aaaaah https://www.instagram.com/p/BAUx8eQtKez/\r

In [ ]:
dftrain=pd.read_csv('train_2kmZucJ.csv')
dftest=pd.read_csv('test_oJQbWVk.csv') 

In [ ]:
def English(Inpdata):
    cleanedArticle1=re.sub(r'[?|$|(),"".@#=><|!]Â&*/',r' ',Inpdata)
    cleanedArticle2=re.sub(r'https?://\S+|www\.\S+',r' ',cleanedArticle1)
    cleanedArticle3=re.sub(r'[^a-z A-Z 0-9]',r' ',cleanedArticle2)
    cleanedArticle4=cleanedArticle3.lower()
    cleanedArticle5=re.sub(r'\b\w{1,2}\b', ' ',cleanedArticle4)
    cleanedArticle6=re.sub(r' +', ' ',cleanedArticle5)
    return(cleanedArticle6)

In [ ]:
dftrain['tweet_new']=dftrain['tweet'].apply(English)

In [ ]:
dftest['tweet_new']=dftest['tweet'].apply(English)

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
def Token_new(inpdata):
    from nltk.tokenize import RegexpTokenizer
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(inpdata)
    return(tokens)

In [ ]:
dftrain['tweet_token']=dftrain['tweet_new'].apply(Token_new)

In [ ]:
dftest['tweet_token']=dftest['tweet_new'].apply(Token_new)

In [ ]:
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer

In [ ]:
def Stem_Stop(Inpdata):
    texts=[]
    en_stop = get_stop_words('en')
    ps=PorterStemmer()
    stopped_tokens = [i for i in Inpdata if not i in en_stop]
    porter_stem=[ps.stem(word) for word in stopped_tokens ]
    texts.append(porter_stem)
    return(texts)

In [ ]:
dftrain['tweet_stem']=dftrain['tweet_token'].apply(Stem_Stop)

In [ ]:
dftest['tweet_stem']=dftest['tweet_token'].apply(Stem_Stop)

In [ ]:
corp_Train=dftrain.tweet_stem
document_Train=[''.join(str(text))for text in corp_Train]

In [ ]:
corp_test=dftest.tweet_stem
document_test=[''.join(str(text))for text in corp_test]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer(ngram_range=(2,2),max_features=300,min_df=7)
X=vectorizer.fit_transform(document_Train)


In [ ]:
Frame_Train=pd.DataFrame(data=X.toarray(),columns=vectorizer.get_feature_names())
Frame_Train.shape

(7920, 300)

In [ ]:
Frame_Train['label']=dftrain['label']

In [ ]:
selected_feat=Frame_Train.columns
Target='label'

In [ ]:
X=Frame_Train[selected_feat].values
y=Frame_Train[Target].values

In [ ]:
 from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
 lda=LinearDiscriminantAnalysis(n_components=min(1,300),)
 reduced_X=lda.fit(X,y).transform(X)


In [ ]:
selected_columns=['LDA-1']
LDA_Frame=pd.DataFrame(data=reduced_X,columns=selected_columns)

In [ ]:
LDA_Frame['Target']=y
LDA_Frame.head()

,LDA-1,Target
0,-1.515410,0
1,-1.051999,0
2,0.495032,0
3,-1.235741,0
4,0.495032,1


In [ ]:
Predictors=['LDA-1']
Target=['Target']

X=LDA_Frame[Predictors].values
y=LDA_Frame[Target].values

In [ ]:
X=np.reshape(X,(-1,1))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=890, test_size=0.3)

In [ ]:
params=  {
           'n_estimators':[50,150,200,300,400,500],
            'learning_rate':[0.1,0.01,0.02,0.2,0.3,0.001],
             'random_state':[5425,748,354,546,2456],
              
           }

In [ ]:
RF=RandomForestClassifier(max_depth=4,n_estimators=100)
bag=AdaBoostClassifier(base_estimator=RF)
GV=GridSearchCV(estimator=bag,param_grid=params,scoring='f1_weighted',cv=5)
model=GV.fit(X,y)
prediction=model.predict(X)


KeyboardInterrupt: ignored

In [ ]:
import numpy as np
import pandas as pd
np.sum(model[s1], axis=0)
trainData=pd.DataFrame()
trainData['col1']=(np.sum(model[s1], axis=0))
trainData['col2']=(np.sum(model[s2], axis=0))
trainData['col3']=(np.sum(model[s3], axis=0))
trainData['col4']=(np.sum(model[s4], axis=0))
newData=trainData.T
newData['label']=labels
newData

In [ ]:

0	1	2	3	4	5	label
col1	-0.028093	0.142314	-0.020950	-0.205912	-0.020556	-0.174518	1
col2	0.035957	0.120630	-0.160122	-0.059978	0.016202	0.037029	0
col3	-0.043079	0.076913	0.019426	-0.198113	-0.047501	-0.004207	1
col4	0.159873	0.162264	-0.161948	0.020268	0.006707	0.026823	0

In [ ]:
from sklearn.neural_network import MLPClassifier
X = trainData.T.values
y = labels
clf = MLPClassifier(solver='lbfgs', random_state=1)
print(clf.fit(X, y))

TestSentence=np.sum(model[s5], axis=0)
clf.predict(TestSentence.reshape(1,-1)) #reshaping since its a single sample and deprication warning

In [ ]:
print('classification report',metrics.classification_report(y,prediction))
print('confusion matrix',metrics.confusion_matrix(y,prediction))
print('F1 score',metrics.f1_score(y,prediction,average='weighted'))

classification report               precision    recall  f1-score   support

           0       0.80      0.98      0.88      5894
           1       0.81      0.29      0.42      2026

    accuracy                           0.80      7920
   macro avg       0.81      0.63      0.65      7920
weighted avg       0.80      0.80      0.76      7920

confusion matrix [[5760  134]
 [1444  582]]
F1 score 0.7631268388954849


In [ ]:
y_test=y[:1953]

In [ ]:
X_Test=vectorizer.fit_transform(document_test)
Reduced_X_Test=lda.fit(X_Test.toarray(),y_test).transform(X_Test.toarray())

In [ ]:
Prediction_Test=model.predict(Reduced_X_Test)

In [ ]:
dftest.head()

,id,tweet,tweet_new,tweet_token,tweet_stem
0,7921,I hate the new #iphone upgrade. Won't let me d...,hate the new iphone upgrade won let download ...,"[hate, the, new, iphone, upgrade, won, let, do...","[[hate, new, iphon, upgrad, won, let, download..."
1,7922,currently shitting my fucking pants. #apple #i...,currently shitting fucking pants apple imac ca...,"[currently, shitting, fucking, pants, apple, i...","[[current, shit, fuck, pant, appl, imac, cashm..."
2,7923,"I'd like to puts some CD-ROMS on my iPad, is t...",like puts some roms ipad that possible yes bu...,"[like, puts, some, roms, ipad, that, possible,...","[[like, put, rom, ipad, possibl, ye, wouldn, b..."
3,7924,My ipod is officially dead. I lost all my pict...,ipod officially dead lost all pictures and vi...,"[ipod, officially, dead, lost, all, pictures, ...","[[ipod, offici, dead, lost, pictur, video, 5so..."
4,7925,Been fighting iTunes all night! I only want th...,been fighting itunes all night only want the m...,"[been, fighting, itunes, all, night, only, wan...","[[fight, itun, night, want, music, paid]]"


In [ ]:
sample_df=pd.DataFrame(dftest, columns=['id'])
sample_df['label']=Prediction_Test

In [ ]:
sample_df.to_csv('sample_df.csv', index=False)